# PROJET DEEP LEARNING

## IMPORTATION DES MODULES

In [16]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary

## PRE-TRAITEMENT DES DONNEES

In [2]:
print('total training damage images: ',len(os.listdir("./train_another/damage")))
print('total validation damage images: ',len(os.listdir("./validation_another/damage")))
print('total test damage images: ',len(os.listdir("./test_another/damage")))
print('total training no damage images: ',len(os.listdir("./train_another/no_damage")))
print('total validation no damage images: ',len(os.listdir("./validation_another/no_damage")))
print('total test no damage images: ',len(os.listdir("./test_another/no_damage")))

total training damage images:  5000
total validation damage images:  1000
total test damage images:  8000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


### Importation des images en séparant en jeu train / test / validation

In [3]:
transform = transforms.Compose([transforms.Resize((150,150)),
                                 transforms.ToTensor()])
batch_size = 20

df_train = datasets.ImageFolder("./train_another",transform=transform)
train_dataloader = DataLoader(dataset=df_train, batch_size=batch_size, shuffle=True, num_workers = 4 , pin_memory = True)

df_test = datasets.ImageFolder("./test_another",transform=transform)
test_dataloader = DataLoader(dataset=df_test, batch_size=batch_size, shuffle=True,num_workers = 4 , pin_memory = True)

df_validation = datasets.ImageFolder("./validation_another",transform=transform)
validation_dataloader = DataLoader(dataset=df_validation, batch_size=batch_size, shuffle=True,num_workers = 4 , pin_memory = True)

Essayons de visualiser les classes de nos images 

In [4]:
df_validation.classes

['damage', 'no_damage']

## MODELISATION

### MODELE 1 : Reproduction du modèle de CAO & CHOE

#### Création de la classe qui va nous permettre de faire notre classification d'images

In [6]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

#### Création de la classe qui va permettre de créer notre premier modèle

In [7]:
class HurricaneClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3,32, kernel_size = (3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64, kernel_size = (3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = (3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(128,128, kernel_size = (3,3)),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(6272, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.ReLU(),
        )
    
    def forward(self, xb):
        return self.network(xb)

#### Création des fonctions qui vont permettre d'évaluer notre modèle

In [20]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    nb_etape = 0
    outputs = []
    for batch in val_loader:
        nb_etape = nb_etape + 1
        outputs.append(model.validation_step(batch))
        if nb_etape == 50:
            break
            
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        nb_etape = 0
        
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            nb_etape = nb_etape+1
            if nb_etape == 10:
                break
                
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history, train_losses

In [22]:
model = HurricaneClassification()
num_epochs = 5
opt_func = torch.optim.Adam
lr = 1e-4
#fitting the model on training data and record the result after each epoch
history, loss = fit(num_epochs, lr, model, train_dataloader, validation_dataloader, opt_func)

Epoch [0], train_loss: 0.6948, val_loss: 0.6929, val_acc: 0.5090
Epoch [1], train_loss: 0.6928, val_loss: 0.6926, val_acc: 0.5020
Epoch [2], train_loss: 0.6928, val_loss: 0.6919, val_acc: 0.4880
Epoch [3], train_loss: 0.6928, val_loss: 0.6929, val_acc: 0.5030
Epoch [4], train_loss: 0.6923, val_loss: 0.6901, val_acc: 0.5040


In [23]:
summary(model,(3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 148, 148]             896
              ReLU-2         [-1, 32, 148, 148]               0
         MaxPool2d-3           [-1, 32, 74, 74]               0
            Conv2d-4           [-1, 64, 72, 72]          18,496
              ReLU-5           [-1, 64, 72, 72]               0
         MaxPool2d-6           [-1, 64, 36, 36]               0
            Conv2d-7          [-1, 128, 34, 34]          73,856
              ReLU-8          [-1, 128, 34, 34]               0
         MaxPool2d-9          [-1, 128, 17, 17]               0
           Conv2d-10          [-1, 128, 15, 15]         147,584
             ReLU-11          [-1, 128, 15, 15]               0
        MaxPool2d-12            [-1, 128, 7, 7]               0
          Flatten-13                 [-1, 6272]               0
          Dropout-14                 [-

In [24]:
plt.plot(loss)
plt.xlabel('Itération')
plt.ylabel('Perte')
plt.show()

: 

: 